<a href="https://colab.research.google.com/github/gauravraidata/IITJ-projects/blob/main/AI_Ass2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Q1

In [3]:
# Sudoku CSP Solver with Backtracking, Heuristics, and Inference

import time
import random
import copy
from collections import defaultdict

# ------------------ CSP Class ------------------
class SudokuCSP:
    def __init__(self, board):
        self.board = board
        self.variables = [(r, c) for r in range(9) for c in range(9) if board[r][c] == 0]
        self.domains = {var: list(range(1, 10)) for var in self.variables}
        self.neighbors = self.build_neighbors()

    def build_neighbors(self):
        neighbors = defaultdict(set)
        for r in range(9):
            for c in range(9):
                box_r, box_c = 3 * (r // 3), 3 * (c // 3)
                related = set()
                related.update([(r, j) for j in range(9) if j != c])
                related.update([(i, c) for i in range(9) if i != r])
                related.update([(i, j) for i in range(box_r, box_r + 3) for j in range(box_c, box_c + 3) if (i, j) != (r, c)])
                neighbors[(r, c)] = related
        return neighbors

    def is_valid(self, var, value, assignment):
        for neighbor in self.neighbors[var]:
            if assignment.get(neighbor) == value:
                return False
        return True

# ------------------ Heuristics ------------------
def select_unassigned_variable(assignment, csp, heuristic="MRV"):
    unassigned = [v for v in csp.variables if v not in assignment]

    if heuristic == "MRV":
        return min(unassigned, key=lambda var: len(csp.domains[var]))
    elif heuristic == "Degree":
        return max(unassigned, key=lambda var: len([n for n in csp.neighbors[var] if n not in assignment]))
    else:
        return unassigned[0]

def order_domain_values(var, csp, assignment):
    return sorted(csp.domains[var], key=lambda val: count_conflicts(var, val, csp, assignment))

def count_conflicts(var, val, csp, assignment):
    return sum(1 for neighbor in csp.neighbors[var]
               if neighbor in csp.domains and val in csp.domains[neighbor])


# ------------------ Inference ------------------
def forward_checking(var, value, csp, assignment, domains):
    local_domains = copy.deepcopy(domains)
    for neighbor in csp.neighbors[var]:
        if neighbor not in assignment and value in local_domains[neighbor]:
            local_domains[neighbor].remove(value)
            if not local_domains[neighbor]:
                return None
    return local_domains

def AC3(csp):
    queue = [(xi, xj) for xi in csp.variables for xj in csp.neighbors[xi]]
    while queue:
        xi, xj = queue.pop(0)
        if revise(csp, xi, xj):
            if not csp.domains[xi]:
                return False
            for xk in csp.neighbors[xi] - {xj}:
                queue.append((xk, xi))
    return True

def revise(csp, xi, xj):
    revised = False
    for x in csp.domains[xi][:]:
        if all(x == y for y in csp.domains[xj]):
            csp.domains[xi].remove(x)
            revised = True
    return revised

# ------------------ Backtracking Solver ------------------
def backtrack(assignment, csp, inference, heuristic):
    if len(assignment) == len(csp.variables):
        return assignment

    var = select_unassigned_variable(assignment, csp, heuristic)
    for value in order_domain_values(var, csp, assignment):
        if csp.is_valid(var, value, assignment):
            assignment[var] = value
            orig_domains = copy.deepcopy(csp.domains)
            csp.domains[var] = [value]

            if inference == "FC":
                new_domains = forward_checking(var, value, csp, assignment, csp.domains)
                if new_domains is None:
                    del assignment[var]
                    csp.domains = orig_domains
                    continue
                csp.domains = new_domains

            result = backtrack(assignment, csp, inference, heuristic)
            if result is not None:
                return result
            del assignment[var]
            csp.domains = orig_domains
    return None

# ------------------ Utility ------------------
def solve_sudoku(board, inference=None, heuristic="MRV"):
    csp = SudokuCSP(board)
    AC3(csp) if inference == "AC3" else None
    assignment = {(r, c): board[r][c] for r in range(9) for c in range(9) if board[r][c] != 0}
    start = time.time()
    result = backtrack(assignment, csp, inference, heuristic)
    end = time.time()
    return result, end - start

def print_board(assignment):
    for i in range(9):
        row = []
        for j in range(9):
            val = assignment.get((i, j), 0)
            row.append(str(val))
        print(" ".join(row))


# ------------------ Example Usage ------------------
example_board = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

if __name__ == "__main__":
    print("Solving Sudoku using Backtracking + FC + MRV")
    solution, duration = solve_sudoku(example_board, inference="FC", heuristic="MRV")
    print_board(solution)
    print(f"Solved in {duration:.4f} seconds")


Solving Sudoku using Backtracking + FC + MRV
5 3 2 6 7 8 1 9 4
6 4 7 1 9 5 3 2 8
1 9 8 2 3 4 5 6 7
8 0 0 0 6 0 9 0 3
4 0 0 8 0 3 7 0 1
7 0 0 0 2 0 4 0 6
0 6 0 0 0 0 2 8 0
0 0 0 4 1 9 6 0 5
0 0 0 0 8 0 0 7 9
Solved in 0.2007 seconds


###Q2

In [4]:
import random
import math
import time

# ---------------------
# Cost function: Number of attacking pairs
# ---------------------
def compute_cost(state):
    conflicts = 0
    n = len(state)
    for i in range(n):
        for j in range(i + 1, n):
            if state[i] == state[j] or abs(state[i] - state[j]) == j - i:
                conflicts += 1
    return conflicts

# ---------------------
# Generate a neighbor: swap two queens
# ---------------------
def get_neighbor(state):
    neighbor = state.copy()
    i, j = random.sample(range(len(state)), 2)
    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor

# ---------------------
# Simulated Annealing Solver
# ---------------------
def simulated_annealing(n=8, T=100.0, T_min=1e-3, alpha=0.99, max_iter=10000, use_HC=False):
    state = list(range(n))
    random.shuffle(state)
    current_cost = compute_cost(state)
    steps = 0
    start_time = time.time()

    while T > T_min and steps < max_iter and current_cost != 0:
        neighbor = get_neighbor(state)
        neighbor_cost = compute_cost(neighbor)
        delta = neighbor_cost - current_cost

        if delta < 0 or (not use_HC and random.random() < math.exp(-delta / T)):
            state = neighbor
            current_cost = neighbor_cost

        T *= alpha
        steps += 1

    end_time = time.time()
    return state, compute_cost(state), steps, end_time - start_time


In [5]:
N = 8

print("=== Simulated Annealing ===")
sa_solution, sa_cost, sa_steps, sa_time = simulated_annealing(N)
print("Final Cost:", sa_cost)
print("Steps:", sa_steps)
print("Time:", f"{sa_time:.4f} sec")
print("Solution:", sa_solution)

print("\n=== Hill Climbing (SA with T=0) ===")
hc_solution, hc_cost, hc_steps, hc_time = simulated_annealing(N, T=1.0, use_HC=True)
print("Final Cost:", hc_cost)
print("Steps:", hc_steps)
print("Time:", f"{hc_time:.4f} sec")
print("Solution:", hc_solution)


=== Simulated Annealing ===
Final Cost: 0
Steps: 331
Time: 0.0025 sec
Solution: [1, 5, 0, 6, 3, 7, 2, 4]

=== Hill Climbing (SA with T=0) ===
Final Cost: 0
Steps: 32
Time: 0.0007 sec
Solution: [0, 6, 3, 5, 7, 1, 4, 2]
